In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress
from datetime import datetime

%matplotlib inline

# Impor the OpenWeatherMap API key
from config import api_key

In [ ]:
#Variables used in API call, oldest date is furthest back we are willing to go
oldest_date = "2014-01-31"
company_symbols = ["TAN",
                    "ENPH",
                    "SEDG",
                    "FSLR",
                    "HYDR",
                    "PLUG",
                    "BLDP",
                    "BE",
                    "FAN",
                    "TAC",
                    "TPIC",
                    "AMSC"]
time_function = "TIME_SERIES_MONTHLY"
base_url = "https://www.alphavantage.co/query?"

company_data = []

for symbol in company_symbols:
    finished_url = f"{base_url}function={time_function}&symbol={symbol}&apikey={api_key}"
    print(f"Processing {symbol}")

    result_data = requests.get(finished_url).json()

    time.sleep(1)

    for monthly in result_data["Monthly Time Series"]:
        monthly_date = time.strptime(monthly, "%Y-%m-%d")
        oldest = time.strptime(oldest_date, "%Y-%m-%d")
        if (monthly_date < oldest):
            continue
        stock_date = monthly
        stock_close = result_data["Monthly Time Series"][monthly]["4. close"]
        company_data.append({"Symbol":symbol,
                            "Date":stock_date,
                            "Closing Price":stock_close})

company_data_df = pd.DataFrame(company_data)
company_data_df


In [14]:
#Add a type column and lable to each stock
company_data_df["Type"] = ""
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "TAN", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "ENPH", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "SEDG", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "FSLR", "Solar", company_data_df["Type"])

company_data_df["Type"] = np.where(company_data_df["Symbol"] == "HYDR", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "PLUG", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "BLDP", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "BE", "Hydrogen", company_data_df["Type"])

company_data_df["Type"] = np.where(company_data_df["Symbol"] == "FAN", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "TAC", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "TPIC", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "AMSC", "Wind", company_data_df["Type"])

#Save the Data pulled to a CSV file
company_data_df.to_csv("data/company_data.csv", index_label="Stock_ID")

In [ ]:
#Read data from the saved CSV file and sort it by symbol and date
read_data_df = pd.read_csv("data/company_data.csv", index_col="Stock_ID")
ordered_data_df = read_data_df.sort_values(by=["Symbol", "Date"])
ordered_data_df

In [16]:
#Function to plot graphs showing stock prices
def plot_stock_prices(symbols,given_data_df,type):
    plt.figure(figsize=(14, 6))

    for symbol in symbols:
        stock_data = given_data_df[given_data_df["Symbol"] == symbol]
        plt.plot(stock_data["Date"], stock_data["Closing Price"], label=symbol)

    plt.title(f"Stock Prices for {type} Companies")
    plt.xlabel("Date")
    plt.ylabel("Closing Price")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [17]:
#Function to plot graphs showing average yearly stock prices
def plot_avg_prices(symbols,given_data_df,type):
    plt.figure(figsize=(8, 6))

    for symbol in symbols:
        stock_data = given_data_df[given_data_df["Symbol"] == symbol]
        plt.plot(stock_data["Year"], stock_data["Avg. Closing Price"], label=symbol)

    plt.title(f"Avg. Yealy Stock Prices for {type} Companies")
    plt.xlabel("Year")
    plt.ylabel("Avg. Closing Price")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [18]:
#Function to plot graphs showing max yearly stock prices
def plot_max_prices(symbols,given_data_df,type):
    plt.figure(figsize=(8, 6))

    for symbol in symbols:
        stock_data = given_data_df[given_data_df["Symbol"] == symbol]
        plt.plot(stock_data["Year"], stock_data["Max Closing Price"], label=symbol)

    plt.title(f"Max Yealy Stock Prices for {type} Companies")
    plt.xlabel("Year")
    plt.ylabel("Max Closing Price")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot Stocks
solar_symbols = ["TAN", "ENPH", "SEDG", "FSLR"]
hydrogen_symbols = ["BLDP", "PLUG", "BE", "HYDR"]
wind_symbols = ["FAN", "TAC", "TPIC", "AMSC"]

plot_stock_prices(solar_symbols,ordered_data_df,"Solar")
plot_stock_prices(hydrogen_symbols,ordered_data_df,"Hydrogen")
plot_stock_prices(wind_symbols,ordered_data_df,"Wind")

In [ ]:
#Yearly Average stock price
yearly_average = []
for symbol in company_symbols:
    starting_year = int(ordered_data_df[ordered_data_df["Symbol"] == symbol]["Date"].str.extract(r"(\d{4})").iloc[0,0])
    for year in range(starting_year, 2025):
        years = ordered_data_df["Date"].str.extract(r"(\d{4})")[0]
        average = ordered_data_df.loc[(ordered_data_df["Symbol"] == symbol) & (years == str(year)), "Closing Price"].mean()
        yearly_average.append({"Symbol":symbol,
                                "Year":year,
                                "Avg. Closing Price":average,
                                "Type":ordered_data_df.loc[ordered_data_df["Symbol"] == symbol,"Type"].iloc[0]})

yearly_average_df = pd.DataFrame(yearly_average)
yearly_average_df  

In [ ]:
#Yearly max stock price
yearly_max = []
for symbol in company_symbols:
    starting_year = int(ordered_data_df[ordered_data_df["Symbol"] == symbol]["Date"].str.extract(r"(\d{4})").iloc[0,0])
    for year in range(starting_year, 2025):
        years = ordered_data_df["Date"].str.extract(r"(\d{4})")[0]
        max = ordered_data_df.loc[(ordered_data_df["Symbol"] == symbol) & (years == str(year)), "Closing Price"].max()
        yearly_max.append({"Symbol":symbol,
                                "Year":year,
                                "Max Closing Price":max,
                                "Type":ordered_data_df.loc[ordered_data_df["Symbol"] == symbol,"Type"].iloc[0]})

yearly_max_df = pd.DataFrame(yearly_max)
yearly_max_df

In [ ]:
#Plot Avg. Yearly Stock Prices
plot_avg_prices(solar_symbols,yearly_average_df,"Solar")
plot_avg_prices(hydrogen_symbols,yearly_average_df,"Hydrogen")
plot_avg_prices(wind_symbols,yearly_average_df,"Wind")

In [ ]:
#Plot Max Yearly Stock Prices
plot_max_prices(solar_symbols,yearly_max_df,"Solar")
plot_max_prices(hydrogen_symbols,yearly_max_df,"Hydrogen")
plot_max_prices(wind_symbols,yearly_max_df,"Wind")